In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 4

Populating the interactive namespace from numpy and matplotlib


## Train dataset

In [2]:
train = pd.read_csv('../data/train.csv')
print(train.columns)
print('Unique locations: %d' % len(train['location'].unique()))
print('train dataset size: %d' % len(train))
train.head()

Index(['id', 'location', 'fault_severity'], dtype='object')
Unique locations: 929
train dataset size: 7381


,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0


## Test data set

In [3]:
test = pd.read_csv('../data/test.csv')
print('test dataset size: %d' % len(test))
test.head()

test dataset size: 11171


,id,location
0,11066,location 481
1,18000,location 962
2,16964,location 491
3,4795,location 532
4,3392,location 600


## Sample solution

In [4]:
sample_submission = pd.read_csv('../data/sample_submission.csv')
sample_submission.head()

,id,predict_0,predict_1,predict_2
0,11066,0,1,0
1,18000,0,1,0
2,16964,0,1,0
3,4795,0,1,0
4,3392,0,1,0


In [5]:
log_feature = pd.read_csv('../data/log_feature.csv')
log_feature['volume'] = log_feature['volume'] / log_feature['volume'].max()
print('Unique features: %d' % len(log_feature['log_feature'].unique()))
print('log_feature dataset size: %d' % len(log_feature))
sparse_feature = log_feature.groupby(['id', 'log_feature']).max()
sparse_feature = sparse_feature.unstack(1).fillna(0)
sparse_feature = sparse_feature['volume'].reset_index()
sparse_feature.head()

Unique features: 386
log_feature dataset size: 58671


log_feature,id,feature 1,feature 10,feature 100,feature 101,feature 102,feature 103,feature 104,feature 105,feature 106,...,feature 90,feature 91,feature 92,feature 93,feature 94,feature 95,feature 96,feature 97,feature 98,feature 99
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
event_type = pd.read_csv('../data/event_type.csv')
print('Unique types: %d' % len(event_type['event_type'].unique()))
print('event_type dataset size: %d' % len(event_type))
event_type.head()

Unique types: 53
event_type dataset size: 31170


,id,event_type
0,6597,event_type 11
1,8011,event_type 15
2,2597,event_type 15
3,5022,event_type 15
4,5022,event_type 11


In [7]:
resource_type = pd.read_csv('../data/resource_type.csv')
print('resource_type dataset size: %d' % len(resource_type))
resource_type.head()

resource_type dataset size: 21076


,id,resource_type
0,6597,resource_type 8
1,8011,resource_type 8
2,2597,resource_type 8
3,5022,resource_type 8
4,6852,resource_type 8


In [8]:
severity_type = pd.read_csv('../data/severity_type.csv')
print('severity_type dataset size: %d' % len(severity_type))
severity_type.head()

severity_type dataset size: 18552


,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1


## Join all dataframes

In [9]:
df = train.merge(sparse_feature, on='id')
# df = train.merge(gs, on='id')
# df = df.merge(event_type, on='id')
# df = df.merge(resource_type, on='id')
# df = df.merge(severity_type, on='id')
print('Train count: %d' % len(train))
print('DataFrame shape: %d, %d' % (len(df), len(df.columns)))
df.head()

Train count: 7381
DataFrame shape: 7381, 389


,id,location,fault_severity,feature 1,feature 10,feature 100,feature 101,feature 102,feature 103,feature 104,...,feature 90,feature 91,feature 92,feature 93,feature 94,feature 95,feature 96,feature 97,feature 98,feature 99
0,14121,location 118,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9320,location 91,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14394,location 152,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8218,location 931,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14804,location 120,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
